## Get insulin sequence

In [40]:
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa
from Bio.Data.IUPACData import protein_letters_3to1

def three_to_one(resname):
    try:
        return protein_letters_3to1[resname.capitalize()]
    except KeyError:
        return "X"

# Load the PDB structure
parser = PDBParser(QUIET=True)
structure = parser.get_structure("insulin", "3i40.pdb")  # use your file name

# Extract residues and convert to 1-letter sequence
sequence = ""
for model in structure:
    for chain in model:
        for residue in chain:
            if is_aa(residue, standard=True):
                try:
                    sequence += three_to_one(residue.resname)
                except KeyError:
                    print(f"Skipping non-standard residue: {residue.resname}")

print("Extracted sequence:", sequence)

Extracted sequence: GIVEQCCTSICSLYQLENYCNFVNQHLCGSHLVEALYLVCGERGFFYTPKA


## Use pepsickle to identify residues that are vulnerable

In [43]:
import subprocess

result = subprocess.run(
    ['conda', 'run', '-n', 'pepsickle-env', 'pepsickle', '-s', sequence],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("Pepsickle Output:\n", result.stdout)
else:
    print(f"Error:\n{result.stderr}")

Pepsickle Output:
 position	residue	cleav_prob	cleaved	protein_id
1	G	0.0101	False	None
2	I	0.0079	False	None
3	V	0.0075	False	None
4	E	0.0134	False	None
5	Q	0.03	False	None
6	C	0.0826	False	None
7	C	0.0467	False	None
8	T	0.1358	False	None
9	S	0.0541	False	None
10	I	0.5724	True	None
11	C	0.1199	False	None
12	S	0.0367	False	None
13	L	0.3271	False	None
14	Y	0.3837	False	None
15	Q	0.0511	False	None
16	L	0.4461	False	None
17	E	0.0674	False	None
18	N	0.0703	False	None
19	Y	0.6955	True	None
20	C	0.1105	False	None
21	N	0.0587	False	None
22	F	0.7819	True	None
23	V	0.1943	False	None
24	N	0.0714	False	None
25	Q	0.1259	False	None
26	H	0.4295	False	None
27	L	0.8347	True	None
28	C	0.2038	False	None
29	G	0.069	False	None
30	S	0.0636	False	None
31	H	0.0765	False	None
32	L	0.2922	False	None
33	V	0.2123	False	None
34	E	0.0707	False	None
35	A	0.3311	False	None
36	L	0.4705	False	None
37	Y	0.7677	True	None
38	L	0.8801	True	None
39	V	0.8811	True	None
40	C	0.2311	False	None
41	G	0.1179	False	None
42	E	0.054

In [44]:
from io import StringIO
import pandas as pd

df = pd.read_csv(StringIO(result.stdout), sep='\t')
print(df.head())

   position residue  cleav_prob  cleaved  protein_id
0         1       G      0.0101    False         NaN
1         2       I      0.0079    False         NaN
2         3       V      0.0075    False         NaN
3         4       E      0.0134    False         NaN
4         5       Q      0.0300    False         NaN


In [45]:
cleavage_risk_sites = df[df['cleaved'] == True]['position']
print(cleavage_risk_sites)

9     10
18    19
21    22
26    27
36    37
37    38
38    39
46    47
48    49
49    50
Name: position, dtype: int64


In [46]:
# Critical insulin recidues for receptor binding from research online
receptor_interface = [12, 16, 24, 25, 26, 19]

In [52]:
from Bio.PDB import PDBParser

structure = PDBParser(QUIET=True).get_structure("insulin", "3i40.pdb")
chains = {}

for model in structure:
    for chain in model:
        res_nums = [residue.id[1] for residue in chain if residue.id[0] == " "]
        chains[chain.id] = res_nums

for chain_id, nums in chains.items():
    print(f"Chain {chain_id}: {min(nums)}–{max(nums)} (residues: {nums})")

Chain A: 1–21 (residues: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])
Chain B: 1–30 (residues: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])


In [59]:
from Bio.PDB import PDBParser, PDBIO, Select

class CleanSelect(Select):
    def accept_residue(self, residue):
        return residue.id[0] == " "  # Only standard residues

parser = PDBParser(QUIET=True)
structure = parser.get_structure("insulin", "3i40.pdb")

io = PDBIO()
io.set_structure(structure)
io.save("clean_3i40_cleaned.pdb", CleanSelect())

In [60]:
residues = set()
with open("clean_3i40_cleaned.pdb") as f:
    for line in f:
        if line.startswith("ATOM"):
            chain = line[21]
            res_no = int(line[22:26].strip())
            residues.add((chain, res_no))

print(sorted(residues))

[('A', 1), ('A', 2), ('A', 3), ('A', 4), ('A', 5), ('A', 6), ('A', 7), ('A', 8), ('A', 9), ('A', 10), ('A', 11), ('A', 12), ('A', 13), ('A', 14), ('A', 15), ('A', 16), ('A', 17), ('A', 18), ('A', 19), ('A', 20), ('A', 21), ('B', 1), ('B', 2), ('B', 3), ('B', 4), ('B', 5), ('B', 6), ('B', 7), ('B', 8), ('B', 9), ('B', 10), ('B', 11), ('B', 12), ('B', 13), ('B', 14), ('B', 15), ('B', 16), ('B', 17), ('B', 18), ('B', 19), ('B', 20), ('B', 21), ('B', 22), ('B', 23), ('B', 24), ('B', 25), ('B', 26), ('B', 27), ('B', 28), ('B', 29), ('B', 30)]


# Design a carrier using RFdiffusion

In [73]:
import os
import subprocess

# Set up paths
project_dir = os.getcwd()  # Should be ProteinDesign/
rf_dir = os.path.join(project_dir, "RFdiffusion")
input_pdb = os.path.join(project_dir, "clean_3i40_cleaned.pdb")
output_dir = os.path.join(project_dir, "rf_output")
os.makedirs(output_dir, exist_ok=True)

# Only use cleavage-prone residues for design
design_site_indices = sorted(set(cleavage_risk_sites.tolist()))
# Make sure cleavage and receptor sets are disjoint
design_site_indices = sorted(set(cleavage_risk_sites.tolist()) - set(receptor_interface))
design_site_indices = [50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]

# Optional: assert that you're not accidentally designing over the interface
assert all(idx not in receptor_interface for idx in design_site_indices), "Some receptor residues are being designed!"

# Prepare RFdiffusion command
cleavage_str = ",".join(map(str, design_site_indices))

contig_str = "[50-70/0/A3-24/71-100]"

rf_cmd = ['conda', 'run', '-n', 'SE3nv',
    "python", "scripts/run_inference.py",
    "--config-path", "../config/inference",
    "--config-name", "base",
    f"+input_pdb={input_pdb}",
    f"+design_site_indices=[{cleavage_str}]",
    f"+output_path={output_dir}",
    "+num_designs=5",
    f"+contigmap.contigs={contig_str}"
]

# Run RFdiffusion
print("Running RFdiffusion...")
result = subprocess.run(rf_cmd, cwd=rf_dir, capture_output=True, text=True)
print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

if result.returncode != 0:
    print("RFdiffusion failed ❌")
else:
    print("RFdiffusion completed ✅")

Running RFdiffusion...
STDOUT:
 [2025-04-04 15:07:27,238][__main__][INFO] - Found GPU with device_name NVIDIA GeForce RTX 3070 Ti. Will run RFdiffusion on NVIDIA GeForce RTX 3070 Ti
Reading models from /home/karthik_pc/projects/ProteinDesign/RFdiffusion/rfdiffusion/inference/../../models
[2025-04-04 15:07:27,238][rfdiffusion.inference.model_runners][INFO] - Reading checkpoint from /home/karthik_pc/projects/ProteinDesign/RFdiffusion/rfdiffusion/inference/../../models/Base_ckpt.pt
This is inf_conf.ckpt_path
/home/karthik_pc/projects/ProteinDesign/RFdiffusion/rfdiffusion/inference/../../models/Base_ckpt.pt
Assembling -model, -diffuser and -preprocess configs from checkpoint
USING MODEL CONFIG: self._conf[model][n_extra_block] = 4
USING MODEL CONFIG: self._conf[model][n_main_block] = 32
USING MODEL CONFIG: self._conf[model][n_ref_block] = 4
USING MODEL CONFIG: self._conf[model][d_msa] = 256
USING MODEL CONFIG: self._conf[model][d_msa_full] = 64
USING MODEL CONFIG: self._conf[model][d_pair]

# Collect results

In [76]:
import os
import shutil

# Set paths
samples_dir = "/home/karthik_pc/projects/ProteinDesign/RFdiffusion/samples"
destination_dir = "/home/karthik_pc/projects/ProteinDesign/all_designs"

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy all design.pdb files from samples
for filename in os.listdir(samples_dir):
    if filename.endswith(".pdb"):
        src = os.path.join(samples_dir, filename)
        dst = os.path.join(destination_dir, filename)
        shutil.copy(src, dst)
        print(f"✅ Copied: {filename}")

print(f"\n📦 All designs collected in: {destination_dir}")

✅ Copied: design_3.pdb
✅ Copied: design_5.pdb
✅ Copied: design_0.pdb
✅ Copied: design_4.pdb
✅ Copied: design_9.pdb
✅ Copied: design_1.pdb
✅ Copied: design_6.pdb
✅ Copied: design_2.pdb
✅ Copied: design_8.pdb
✅ Copied: design_7.pdb

📦 All designs collected in: /home/karthik_pc/projects/ProteinDesign/all_designs


## View design

In [1]:
import nglview as nv

view = nv.show_file("/home/karthik_pc/projects/ProteinDesign/all_designs/design_0.pdb")
view

NGLWidget()